![alt text](images/uspas.png)
# VUV and X-ray Free Electron Lasers
# Lab Day 3: Running Genesis with lume-genesis
#### In this session, we will use lume-genesis to do the following:
- load input files
- change input files
- run a genesis simulation
- load output results
- plot output results

The Genesis manual can be found here: http://genesis.web.psi.ch/download.html   
Parts of this notebook taken from lume-genesis examples here: https://github.com/slaclab/lume-genesis/tree/master/examples
##### Authors: N. Neveu, P. Anisimov, D. Nguyen, Y.S. Li
----------

# 1. Import functions we will be using



In [ ]:
import os
from bokeh.plotting import figure, output_file, show, output_notebook

# from lume-genesis
from genesis import Genesis
from genesis import parsers, lattice
from genesis.parsers import parse_beam_file
from genesis.writers import write_beam_file

----

# 2. Load a lattice file

Genesis lattice files consist of components in the beamline (from manual):

- AW - Main magnetic field (undulator)
- AD - Drift section
- QF - Quadrupole strength
- QX - Quadrupole offset in x
- QY - Quadrupole offset in y
- SL - Solenoid strength
- CX - Corrector strength in x
- CY - Corrector strength in y

In [ ]:
# Read genesis lattice file as elements and parameters
genesis_lat = 'examples/lcls_short.lat'
lat = parsers.parse_genesis_lattice(genesis_lat)
lat.keys()

We have loaded two types of information: 
- eles  = elements
- param = parameters 

Information included in the element descriptions: 
- strength = magnet strength
- L = length of element
- d = distance to previous element

Run the next two commands to access and see the information stored in each key

In [ ]:
lat['eles']

In [ ]:
lat['param']

In [ ]:
# You can index through elements like a normal dict: 
lat['eles'][:3]

# 3. Run a simple genesis file

In [ ]:
# lume-genesis will create and use a temp directory unless a working dir is specified.
rundir = os.getcwd()+'/test_run' 
if not os.path.exists(rundir):
    os.makedirs(rundir)

In [ ]:
# Initialize the run, make genesis object with some template input file
G = Genesis('examples/lcls_short.in', verbose=True, workdir=rundir, use_tempdir=False)

We can now look at what is in the input file and make changes.   
All values in G are parameters in the input file.   
Take a look at the information from the input file in the following cells:

In [ ]:
# All input are fields from input file
# if no value in input file template, default is filled in
G.input.keys()

In [ ]:
# list of elements and parameters
# s = end of elements, final positions
# lume-genesis fills in empty spaces for overlap
G.input['lattice']

In [ ]:
# These can also be called w/o key notation:
G.beam     # no beam is loaded yet (empty)
G.lattice  # gives same output as call in last cell
G.param

# Pick a few parameters each, and find them in the manual.
# Summarize what they are used for. 

In [ ]:
# Simulation inputs can be changed before running the simulation
G['nslice'] = 100

# # Turn on field output
# G['idmpfld'] = 1
# # Turn on particle output
# G['idmppar'] = 1
# G['npart'] = 2048
# # Turn on history
# G['ippart'] = 10
# G['ipradi'] = 0
# # Again, these keys come from the input file

In [ ]:
# You can save the updated input file
G.write_input()

In [ ]:
# Run genesis with default lattice 
G.run()

In [ ]:
# Output data is now saved in G object
G.output.keys()

In [ ]:
G.output['run_info']

In [ ]:
# G.output['param'] # should be similar or same as input params
# some numbers filled in during run, slight changes from input

In [ ]:
# G.output['data'] # all the output data

In [ ]:
# G.write_wavefront()

In [ ]:
# These are the available data
G.output['data'].keys()
for key, val in G.output['data'].items():
    print(key, val.shape)

# fld - not parsed here, field history file (can get huge)
# par - can get very large if many slices
# dlf (wavefront), dpa (phase space) are final field and particle files

----

# 4. Archive the data

Save your data to an h5 file, that way you can reload with lume-genesis easily, and only have to save one file.  
You can return to your data after closing jupyterlab, and don't need to leave the browser window open.

In [ ]:
# afile = G.archive()
# # if this is taking a long time, maybe data is too big

In [ ]:
# G2 = Genesis()
# G2.load_archive(afile)

# # Check that all output data are the same
# for k in G.output['data']:
#     print(k, np.all(G.output['data'][k]==G2.output['data'][k]))

----

# 5. Plot the simulation data

In [ ]:
# Get z values in sim
zlist = G.output['data']['z'] # 1D array
zlist.shape

In [ ]:
# Get power. This is a 2d array of: slice, z
power = G.output['data']['power']
power.shape

In [ ]:
# Using bokeh plotting for interactive plot
output_notebook(verbose=False, hide_banner=True)
p = figure(plot_width=800, plot_height=400, y_axis_type='log')

# add a line renderer
p.line(zlist, power[0], line_width=2)

p.xaxis.axis_label = 'Z (M)'
p.yaxis.axis_label = 'power (W)'

show(p)

# X. Loading input beam distributions

----